In [1]:
from argparse import ArgumentParser

from datasets import Priv_NAMES as DATASET_NAMES
from models import get_all_models

from utils.Server import train, local_evaluate


import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


def parse_args():
    parser = ArgumentParser(description='You Only Need Me', allow_abbrev=False)
    parser.add_argument('--device_id', type=int, default=0, help='The Device Id for Experiment')

    # Communication - epochs
    parser.add_argument('--communication_epoch', type=int, default=4,
                        help='The Communication Epoch in Federated Learning')
    parser.add_argument('--local_epoch', type=int, default=5, help='The Local Epoch for each Participant')

    # Participants info
    #TODO: LINK NUMBER OF PARTICIPANTS WITH SIMULATION
    parser.add_argument('--parti_num', type=int, default=None, help='The Number for Participants. If "None" will be setted as the sum of values described in --domain')
    parser.add_argument('--online_ratio', type=float, default=1, help='The Ratio for Online Clients')

    # Data parameters
    parser.add_argument('--dataset', type=str, default='fl_leaks', choices=DATASET_NAMES, help='Which scenario to perform experiments on.')
    parser.add_argument('--experiment_id', type=str, default='PIPELINE_PYTHON', help='Which scenario the experiment is for.')
    parser.add_argument('--domains', type=dict, default={
                                                        'Graeme': 5,
                                                        # 'Balerma': 3,
                                                        },
                        help='Domains and respective number of participants.')

    ## Time series preprocessing
    parser.add_argument('--interval_agg', type=int, default=3 * 60 ** 2,
                        help='Agregation interval (seconds) of time series')
    parser.add_argument('--window_size', type=int, default=200, help='Rolling window length')

    # Model (AER) parameters
    parser.add_argument('--input_size', type=int, default=5, help='Number of sensors')  #TODO adaptar
    parser.add_argument('--output_size', type=int, default=5, help='Shape output - dense layer')
    parser.add_argument('--lstm_units', type=int, default=30,
                        help='Number of LSTM units (the latent space will have dimension 2 times bigger')

    # Federated parameters
    parser.add_argument('--model', type=str, default='fedavg', help='Federated Model name.', choices=get_all_models()) #fedavg

    parser.add_argument('--structure', type=str, default='homogeneity')

    parser.add_argument('--pri_aug', type=str, default='weak',  # weak strong
                        help='Augmentation for Private Data')
    parser.add_argument('--learning_decay', type=bool, default=False, help='The Option for Learning Rate Decay')
    parser.add_argument('--averaging', type=str, default='weight', help='The Option for averaging strategy')

    parser.add_argument('--infoNCET', type=float, default=0.02, help='The InfoNCE temperature')
    parser.add_argument('--T', type=float, default=0.05, help='The Knowledge distillation temperature')
    parser.add_argument('--weight', type=int, default=1, help='The Weigth for the distillation loss')

    # torch.set_num_threads(4)
    # def add_management_args(parser: ArgumentParser) -> None:
    #     parser.add_argument('--csv_log', action='store_true',
    #                         help='Enable csv logging',default=False)
    #
    # add_management_args(parser)
    #
    args, unknown = parser.parse_known_args()

    if args.parti_num is None:
        args.parti_num = sum(args.domains.values())
    #
    # best = best_args[args.dataset][args.model]
    #
    # for key, value in best.items():
    #     setattr(args, key, value)
    #
    # if args.seed is not None:
    #     set_random_seed(args.seed)

    return args

In [2]:
from datasets.utils import FederatedDataset
from models.utils.federated_model import FederatedModel
from argparse import Namespace


def train(model: FederatedModel, private_dataset: FederatedDataset, scenario: str,
          args: Namespace) -> None:
    # if args.csv_log:
    #     csv_writer = CsvWriter(args, private_dataset)

    priv_train_loaders = private_dataset.get_data_loaders(scenario=scenario)

    private_train_loaders = []
    for loader in priv_train_loaders:
        private_train_loaders.append(loader['X'])

    model.trainloaders = private_train_loaders  # TODO REVER ISSO!!!

    if hasattr(model, 'ini'):
        model.ini()


    Epoch = args.communication_epoch
    for epoch_index in range(Epoch):
        model.epoch_index = epoch_index
        if hasattr(model, 'loc_update'):
            epoch_loc_loss_dict = model.loc_update(private_train_loaders)

        print(10 * '**--')
        print('SEM AGREGAÇÃO')

        aux_latent = local_evaluate(model=model, train_dl=priv_train_loaders, private_dataset=private_dataset,
                                    group_detections=False)

        # print(10*'**--')
        # print('COM AGREGAÇÃO')
        # local_evaluate(model = model, train_dl = priv_train_loaders[1], df_results = df_results, group_detections = True)

    return priv_train_loaders, aux_latent


In [3]:
from datasets import get_private_dataset
from models import get_model

args = parse_args()

results = {}

priv_dataset = get_private_dataset(args)

backbones_list = priv_dataset.get_backbone(parti_num=args.parti_num,
                                           names_list=None,
                                           n_series=args.input_size)

model = get_model(backbones_list, args, priv_dataset)

# priv_dataset.EXP_ID = ['Drift_PIPELINE_ALERNATIVE']
# priv_dataset.DOMAINS_LIST = ['Balerma']

priv_train_loaders, aux_latent = train(model=model,
                                       private_dataset=priv_dataset,
                                       scenario='AutoScenario_1',
                                       args=args)

Training client 4
Epoch 1/5 - Loss: 5.2863
Epoch 2/5 - Loss: 4.6354
Epoch 3/5 - Loss: 4.3699
Epoch 4/5 - Loss: 4.0852
Epoch 5/5 - Loss: 3.8697
epochs: 5 / total 5
Training client 1
Epoch 1/5 - Loss: 6.3705
Epoch 2/5 - Loss: 4.1731
Epoch 3/5 - Loss: 4.0533
Epoch 4/5 - Loss: 3.9643
Epoch 5/5 - Loss: 3.8599
epochs: 5 / total 5
Training client 2
Epoch 1/5 - Loss: 5.7498
Epoch 2/5 - Loss: 5.3386
Epoch 3/5 - Loss: 5.0167
Epoch 4/5 - Loss: 4.6497
Epoch 5/5 - Loss: 4.2882
epochs: 5 / total 5
Training client 3
Epoch 1/5 - Loss: 4.9732
Epoch 2/5 - Loss: 4.4394
Epoch 3/5 - Loss: 4.2137
Epoch 4/5 - Loss: 3.9712
Epoch 5/5 - Loss: 3.7469
epochs: 5 / total 5
Training client 0
Epoch 1/5 - Loss: 4.0175
Epoch 2/5 - Loss: 3.6365
Epoch 3/5 - Loss: 3.4073
Epoch 4/5 - Loss: 3.1984
Epoch 5/5 - Loss: 3.0768
epochs: 5 / total 5
**--**--**--**--**--**--**--**--**--**--
SEM AGREGAÇÃO
Acc: 0.7604	Prec: 0.1139	Rec: 0.0659	F1: 0.0835
Acc: 0.7911	Prec: 0.4360	Rec: 0.8898	F1: 0.5853
Acc: 0.7744	Prec: 0.0000	Rec: 0.00

In [4]:
priv_train_loaders[0].keys()

dict_keys(['X_raw', 'X', 'X_norm', 'index', 'y', 'X_index', 'y_index', 'ry_hat', 'y_hat', 'fy_hat', 'x_lat', 'errors', 'results', 'anomalies', 'metrics'])

In [5]:
dl = model.nets_list[0].create_dataloader(X = priv_train_loaders[0]['X'])

In [6]:
for d in b:
    print(d.shape)

NameError: name 'b' is not defined

In [ ]:
len(dl) * 4

In [ ]:
len(priv_train_loaders[0]['X_index'])

In [12]:
nets_list = priv_dataset.get_backbone(parti_num=args.parti_num,
                                           names_list=None,
                                           n_series=args.input_size)

In [7]:
def agg_func(protos):
    """
    Returns the average of the weights.
    """

    for [label, proto_list] in protos.items():
        if len(proto_list) > 1:
            proto = 0 * proto_list[0].data
            for i in proto_list:
                proto += i.data
            protos[label] = proto / len(proto_list)
        else:
            protos[label] = proto_list[0]

    return protos


def calculate_infonce(f_now, f_pos, f_neg):
    device = 'cpu'
    f_proto = torch.cat((f_pos, f_neg), dim=0)
    l = torch.cosine_similarity(f_now, f_proto, dim=1)
    l = l / 0.02

    exp_l = torch.exp(l)
    exp_l = exp_l.view(1, -1)
    pos_mask = [1 for _ in range(f_pos.shape[0])] + [0 for _ in range(f_neg.shape[0])]
    pos_mask = torch.tensor(pos_mask, dtype=torch.float).to(device)
    pos_mask = pos_mask.view(1, -1)
    # pos_l = torch.einsum('nc,ck->nk', [exp_l, pos_mask])
    pos_l = exp_l * pos_mask
    sum_pos_l = pos_l.sum(1)
    sum_exp_l = exp_l.sum(1)
    infonce_loss = -torch.log(sum_pos_l / sum_exp_l)
    return infonce_loss

def hierarchical_info_loss(f_now, label, all_f, mean_f, all_global_protos_keys):
    device = 'cpu'
    print("\n=== DEBUGGING INFO ===")

    # Print type and shape of all_f elements
    print("Type of all_f:", type(all_f))
    if isinstance(all_f, list):
        print(f"all_f contains {len(all_f)} elements.")
        for i, item in enumerate(all_f):
            print(f"Element {i}: Type={type(item)}, Shape={item.shape if hasattr(item, 'shape') else 'N/A'}")
    elif isinstance(all_f, torch.Tensor):
        print(f"all_f is a tensor with shape: {all_f.shape}")
    elif isinstance(all_f, np.ndarray):
        print(f"all_f is a numpy array with shape: {all_f.shape}")

    # Print type and shape of all_global_protos_keys
    print("\nType of all_global_protos_keys:", type(all_global_protos_keys))
    if isinstance(all_global_protos_keys, torch.Tensor):
        print(f"Shape of all_global_protos_keys torch.Tensor: {all_global_protos_keys.shape}")
    elif isinstance(all_global_protos_keys, np.ndarray):
        print(f"Shape of all_global_protos_keys np.array: {all_global_protos_keys.shape}")

    # Print label info
    print("\nType of label:", type(label))
    if isinstance(label, torch.Tensor):
        print(f"Label torch.Tensor value: {label} {label.item()}, Shape: {label.shape}")
    elif isinstance(label, np.ndarray):
        print(f"Label np.ndarray value: {label}, Shape: {label.shape}")


    print("\nType of meanf:", type(mean_f))

    print(f"meanf torch.Tensor value: Shape: {len(mean_f)}")


    # Check boolean indexing
    try:
        indices = (all_global_protos_keys == label.item()).nonzero()
        print(f"\nNumber of matching indices: {len(indices)}")
        if len(indices) > 0:
            print(f"First matching index: {indices[0]}")
            print(f"Indexes: {indices}")
    except Exception as e:
        print("\nError while checking indices:", e)

    print("\n======================\n")

    f_pos = np.array(all_f)[all_global_protos_keys == label.item()][0].to(device)
    f_neg = torch.cat(list(np.array(all_f)[all_global_protos_keys != label.item()])).to(device)

    f_pos = [f for i, f in enumerate(all_f) if all_global_protos_keys[i] == label.item()][0].to(device)
    f_neg = torch.cat([f for i, f in enumerate(all_f) if all_global_protos_keys[i] != label.item()]).to(device)
    xi_info_loss = calculate_infonce(f_now, f_pos, f_neg)


    # mean_f_pos = np.array(mean_f)[all_global_protos_keys == label.item()][0].to(device)
    mean_f_pos = [f for i, f in enumerate(all_f) if all_global_protos_keys[i] == label.item()][0].to(device)
    mean_f_pos = mean_f_pos.view(1, -1)
    # COMENTAR AQUI AQUI O DEBUG

    f_idx = np.where(all_global_protos_keys == label.item())[0][0]
    f_pos = all_f[f_idx].to(device)
    f_neg = torch.cat([f for i, f in enumerate(all_f) if i != f_idx]).to(device)
    xi_info_loss = calculate_infonce(f_now, f_pos, f_neg)

    mean_f_pos = mean_f[f_idx].to(device)

    # mean_f_neg = torch.cat(list(np.array(mean_f)[all_global_protos_keys != label.item()]), dim=0).to(device)
    # mean_f_neg = mean_f_neg.view(9, -1)

    loss_mse = nn.MSELoss()
    cu_info_loss = loss_mse(f_now, mean_f_pos)

    hierar_info_loss = xi_info_loss + cu_info_loss
    return hierar_info_loss

In [24]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import copy
# from utils.args import *
from models.utils.federated_model import FederatedModel
import torch
from utils.finch import FINCH
import numpy as np


global_protos = {} # lista #classes prototipos globais
local_protos = {} # dict #clientes : lista #classes prototipos locais
local_epoch = 3

net = nets_list[0]
train_loader = priv_train_loaders[0]['X']

device = 'cpu'

net = net.to(device)
optimizer = net.optimizer_class(net.parameters(), lr=net.learning_rate, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
criterion.to(device)

if len(global_protos) != 0:
    all_global_protos_keys = np.array(list(global_protos.keys()))
    all_f = []
    mean_f = []
    for protos_key in all_global_protos_keys:
        temp_f = global_protos[protos_key]
        temp_f = torch.cat(temp_f, dim=0).to(device)
        all_f.append(temp_f.cpu())
        mean_f.append(torch.mean(temp_f, dim=0).cpu())
    all_f = [item.detach() for item in all_f]
    mean_f = [item.detach() for item in mean_f]

iterator = tqdm(range(local_epoch))
for iter in iterator:
    agg_protos_label = {}
    for batch_idx, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        labels = labels.to(device)
        f = net.features(images)
        outputs = net.classifier(f)

        lossCE = criterion(outputs, labels)

        if len(global_protos) == 0:
            loss_InfoNCE = 0 * lossCE
        else:
            i = 0
            loss_InfoNCE = None

            for label in labels:
                if label.item() in global_protos.keys():
                    f_now = f[i].unsqueeze(0)
                    loss_instance = hierarchical_info_loss(f_now, label, all_f, mean_f, all_global_protos_keys)

                    if loss_InfoNCE is None:
                        loss_InfoNCE = loss_instance
                    else:
                        loss_InfoNCE += loss_instance
                i += 1
            loss_InfoNCE = loss_InfoNCE / i
        loss_InfoNCE = loss_InfoNCE

        loss = lossCE + loss_InfoNCE
        loss.backward()
        iterator.desc = "Local Pariticipant %d CE = %0.3f,InfoNCE = %0.3f" % (lossCE, loss_InfoNCE)
        optimizer.step()

        if iter == local_epoch - 1:
            for i in range(len(labels)):
                if labels[i].item() in agg_protos_label:
                    agg_protos_label[labels[i].item()].append(f[i, :])
                else:
                    agg_protos_label[labels[i].item()] = [f[i, :]]

agg_protos = agg_func(agg_protos_label)

# # self.local_history[index].append(copy.deepcopy(agg_protos))
# local_history[index].append({
#                                 key: tensor.detach().cpu().numpy()
#                                 for key, tensor in agg_protos.items()
#                             })
#
# local_protos[index] = agg_protos

  0%|          | 0/3 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [9]:
import numpy as np
labels_df = priv_dataset.get_labels()['Graeme']['AutoScenario_1']
labels_dl = labels_df[labels_df['timestamp'].isin(priv_train_loaders[0]['X_index'])]
labels_dl = np.array(labels_dl['label'])

labels_dl.shape

(1481,)

In [10]:
from utils.Server import find_anomalies, evaluate_predictions
import pandas as pd
from backbone.AER import score_anomalies

def process_anomalies_proto(dl, df_anomalies, labels_df):
    labels_df = labels_df.copy()
    labels_df['timestamp'] = labels_df['timestamp'].astype(float)
    y_pred = np.zeros_like(labels_df['timestamp'], dtype=int)

    for _, row in df_anomalies.iterrows():
        mask = (labels_df['timestamp'] >= row['start']) & (labels_df['timestamp'] <= row['end'])
        y_pred[mask] = 1

    labels_df['y_pred'] = y_pred
    dl['results'] = labels_df
    dl['anomalies'] = df_anomalies

    if dl.get('metrics') is None:
        dl['metrics'] = []

    dl['metrics'].append(evaluate_predictions(labels_df))
    return y_pred



def compute_errors_proto(X, ry_hat, y_hat, fy_hat):


    # Convert to numpy if they're PyTorch tensors
    if hasattr(X, 'detach'):
        X = X.detach().cpu().numpy()
    if hasattr(ry_hat, 'detach'):
        ry_hat = ry_hat.detach().cpu().numpy()
    if hasattr(y_hat, 'detach'):
        y_hat = y_hat.detach().cpu().numpy()
    if hasattr(fy_hat, 'detach'):
        fy_hat = fy_hat.detach().cpu().numpy()

    input_window = 200
    aux_errors = []

    for channel in range(X.shape[2]):
        errors = score_anomalies(
            y = X[:, :, channel].reshape(-1, input_window, 1),
            ry_hat = ry_hat[:, channel].reshape(-1, 1),
            y_hat = y_hat[:, :, channel].reshape(-1, input_window - 2, 1),
            fy_hat = fy_hat[:, channel].reshape(-1, 1),
            smoothing_window = 0.01,
            smooth = True,
            mask = True,
            comb = 'mult',
            lambda_rec = 0.5,
            rec_error_type = "dtw"
        )
        aux_errors.append(errors)

    return aux_errors


def local_evaluate_proto(net,
                   dataloader,
                   labels) -> list:

    dataloader['ry_hat'], dataloader['y_hat'], dataloader['fy_hat'], dataloader['x_lat'] = net.predict(dataloader['X'])
    dataloader['errors'] = compute_errors_proto(dataloader['X'], dataloader['ry_hat'], dataloader['y_hat'], dataloader['fy_hat'])

    aux_anomaly = []
    for error in dataloader['errors']:
        detections = find_anomalies(
            errors=error,
            index=dataloader['index'],
            window_size_portion=0.35,
            window_step_size_portion=0.10,
            fixed_threshold=True,
            inverse=True,
            anomaly_padding=50,
            lower_threshold=True
        )
        if len(detections):
            aux_anomaly.append(detections)


    df_anomalies = pd.DataFrame(np.vstack(aux_anomaly), columns=['start', 'end', 'severity'])

    y_pred = process_anomalies_proto(dataloader, df_anomalies, labels)

    return y_pred, dataloader['x_lat'].copy()

In [15]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import copy
# from utils.args import *
from models.utils.federated_model import FederatedModel
import torch
from utils.finch import FINCH
import numpy as np


global_protos = {} # lista #classes prototipos globais
local_protos = {} # dict #clientes : lista #classes prototipos locais
local_epoch = 3

net = nets_list[0]
train_loader = priv_train_loaders[0]['X']

device = 'cpu'

net = net.to(device)
optimizer = net.optimizer_class(net.parameters(), lr=net.learning_rate, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
criterion.to(device)

if len(global_protos) != 0:
    all_global_protos_keys = np.array(list(global_protos.keys()))
    all_f = []
    mean_f = []
    for protos_key in all_global_protos_keys:
        temp_f = global_protos[protos_key]
        temp_f = torch.cat(temp_f, dim=0).to(device)
        all_f.append(temp_f.cpu())
        mean_f.append(torch.mean(temp_f, dim=0).cpu())
    all_f = [item.detach() for item in all_f]
    mean_f = [item.detach() for item in mean_f]

iterator = tqdm(range(local_epoch))
for iter in iterator:
    agg_protos_label = {}
    # for batch_idx, (images, labels) in enumerate(zip(train_loader, labels_dl)):
    #     images = images.reshape(-1, images.shape[0], images.shape[1])
    images = priv_train_loaders[0]
    labels = labels_dl

    optimizer.zero_grad()

    # images = images.to(device)

    model_outputs, f = local_evaluate_proto(net=net, dataloader=images, labels=labels_df)

    # Get matching predictions as tensor (not numpy!)
    # Make sure this slicing doesn't detach gradients
    mask = labels_df['timestamp'].isin(priv_train_loaders[0]['X_index'])
    outputs = model_outputs[mask.values]  # assuming model_outputs is a tensor

    # Ensure labels are tensors and on the same device
    labels_tensor = torch.tensor(labels_df[mask]['label'].values, dtype=torch.long)

    # lossCE = criterion(outputs, labels_tensor)


    if len(global_protos) == 0:
        loss_InfoNCE = 0 #* lossCE
    else:
        i = 0
        loss_InfoNCE = None

        for label in labels:
            if label.item() in global_protos.keys():
                f_now = f[i].unsqueeze(0)
                loss_instance = hierarchical_info_loss(f_now, label, all_f, mean_f, all_global_protos_keys)

                if loss_InfoNCE is None:
                    loss_InfoNCE = loss_instance
                else:
                    loss_InfoNCE += loss_instance
            i += 1
        loss_InfoNCE = loss_InfoNCE / i
    loss_InfoNCE = loss_InfoNCE

    # loss = lossCE + loss_InfoNCE
    # loss.backward()
    iterator.desc = "Local Pariticipant,InfoNCE = %0.3f" % (loss_InfoNCE)
    optimizer.step()

    if iter == local_epoch - 1:
        for i in range(len(labels)):
            if labels[i].item() in agg_protos_label:
                agg_protos_label[labels[i].item()].append(f[i, :])
            else:
                agg_protos_label[labels[i].item()] = [f[i, :]]

agg_protos = agg_func(agg_protos_label)

# # self.local_history[index].append(copy.deepcopy(agg_protos))
# local_history[index].append({
#                                 key: tensor.detach().cpu().numpy()
#                                 for key, tensor in agg_protos.items()
#                             })
#
# local_protos[index] = agg_protos

Local Pariticipant,InfoNCE = 0.000:  33%|███▎      | 1/3 [00:07<00:14,  7.03s/it]

Acc: 0.7574	Prec: 0.2104	Rec: 0.1689	F1: 0.1874


Local Pariticipant,InfoNCE = 0.000:  67%|██████▋   | 2/3 [00:13<00:06,  6.41s/it]

Acc: 0.7574	Prec: 0.2104	Rec: 0.1689	F1: 0.1874


Local Pariticipant,InfoNCE = 0.000: 100%|██████████| 3/3 [00:19<00:00,  6.34s/it]

Acc: 0.7574	Prec: 0.2104	Rec: 0.1689	F1: 0.1874


TypeError: unsupported operand type(s) for *: 'int' and 'memoryview'

In [21]:
len(agg_protos_label[0]), len(agg_protos_label[1])

(1219, 262)

In [62]:
priv_train_loaders[0].keys()

dict_keys(['X_raw', 'X', 'X_norm', 'index', 'y', 'X_index', 'y_index', 'ry_hat', 'y_hat', 'fy_hat', 'x_lat', 'errors', 'results', 'anomalies', 'metrics'])

In [92]:
labels.shape, outputs.shape, labels_df.shape

(torch.Size([1481]), torch.Size([5041]), (5041, 2))

In [98]:
priv_train_loaders[0]['X_index'].shape

(1481,)

In [88]:
outputs

array([1, 1, 1, ..., 0, 0, 0])

In [77]:
images['errors']

[array([1.        , 1.15695572, 1.15695572, ..., 1.73944207, 1.4453733 ,
        1.41214692]),
 array([1.        , 1.15924423, 1.15924423, ..., 1.64210192, 1.38039943,
        1.35471378]),
 array([1.04918256, 1.17327834, 1.17327834, ..., 1.22865496, 1.27967513,
        1.12559913]),
 array([1.00035751, 1.12942248, 1.12942248, ..., 1.67877374, 1.68171828,
        1.48732998]),
 array([1.        , 1.09400447, 1.09400447, ..., 1.1662555 , 1.20078226,
        1.08364716])]